In [1]:
# Model agnostic 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
from typing import Optional, List, Callable, Dict, Any, List
from pathlib import Path
from utils import Helm  # custom model for data handling/model trianing

from tabpfn import TabPFNClassifier, TabPFNRegressor

This version of Feyn and the QLattice is available for academic, personal, and non-commercial use. By using the community version of this software you agree to the terms and conditions which can be found at https://abzu.ai/eula.

In [2]:
# Get the directory this file lives in
nb_dir = Path.cwd() # notebook directory
project_root = nb_dir.parents[0] # project directory
data_path = project_root / "datasets" / "processed_well_data.csv"

includ_cols = ['Dia', 'Dev(deg)','Area (m2)', 'z','GasDens','LiquidDens', 'P/T','friction_factor', 'critical_film_thickness']
D = Helm(path=data_path, includ_cols=includ_cols, test_size=0.20)


In [4]:
def transformer(
        hparams: Dict[str, Any]
):
    transformer = TabPFNRegressor()

    return transformer

hparam_grid = {
            "devices":   ["cpu"],
        }

# train model and optimize hyperparameters via grid search 
trained_model = D.evolv_model(build_model=transformer, hparam_grid=hparam_grid, k_folds=5)




Training model and optimizing hyperparameters via k-fold CV...
/opt/anaconda3/envs/droplet_film/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/droplet_film/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/droplet_film/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/droplet_film/lib/python3.10/site-packages/sklearn/utils/valida